In [1]:
import torch
import cv2
import numpy as np
from PIL import Image
import imageio
import os
from pathlib import Path
import gc
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video
import torchvision.transforms as transforms
from IPython.display import HTML, display
import ipywidgets as widgets
from io import BytesIO
import base64

In [12]:
# سلول 1: تنظیمات بهینه‌سازی حافظه
import torch
import gc
import psutil
import os

class OptimizedVideoConfig:
    # تنظیمات رزولیوشن (کاهش یافته برای RTX 4050)
    OUTPUT_WIDTH = 720       # کاهش از 1080 به 720
    OUTPUT_HEIGHT = 1280     # کاهش از 1920 به 1280
    
    # تنظیمات زمان (کاهش یافته)
    DURATION_SECONDS = 3     # کاهش از 5 به 3 ثانیه
    FPS = 6                  # حفظ همان FPS
    
    # تنظیمات حافظه (بهینه‌سازی شده)
    ENABLE_CPU_OFFLOAD = True
    ENABLE_MEMORY_EFFICIENT = True
    ENABLE_SEQUENTIAL_OFFLOAD = True
    USE_FP16 = True
    
    # تنظیمات جدید برای کاهش مصرف حافظه
    DECODE_CHUNK_SIZE = 2    # کاهش از 8 به 2
    ENABLE_ATTENTION_SLICING = True
    LOW_VRAM_MODE = True
    
    # تنظیمات کیفیت (بهینه‌سازی شده)
    UPSCALE_METHOD = "bicubic"  # تغییر از realesrgan به bicubic
    GUIDANCE_SCALE = 7.5        # کاهش از 10.0
    MOTION_BUCKET_ID = 127
    NOISE_AUG_STRENGTH = 0.02
    
    # تنظیمات دیگر
    SEED = 42
    SAVE_FRAMES = False
    OUTPUT_FORMAT = "mp4"
    
    # تنظیمات پردازش batch
    BATCH_SIZE = 1
    MAX_MEMORY_ALLOCATED = 0.8  # استفاده از 80% حافظه GPU

config = OptimizedVideoConfig()

# محاسبه تعداد فریم‌ها
TOTAL_FRAMES = int(config.DURATION_SECONDS * config.FPS)
print(f"📊 تنظیمات بهینه‌سازی شده:")
print(f"   رزولیوشن نهایی: {config.OUTPUT_WIDTH}x{config.OUTPUT_HEIGHT}")
print(f"   مدت زمان: {config.DURATION_SECONDS} ثانیه")
print(f"   تعداد فریم: {TOTAL_FRAMES}")
print(f"   FPS: {config.FPS}")
print(f"   Decode chunk size: {config.DECODE_CHUNK_SIZE}")

# بررسی حافظه GPU
def check_gpu_memory():
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"   حافظه GPU: {gpu_memory:.1f} GB")
        return gpu_memory
    return 0

gpu_memory = check_gpu_memory()


📊 تنظیمات بهینه‌سازی شده:
   رزولیوشن نهایی: 720x1280
   مدت زمان: 3 ثانیه
   تعداد فریم: 18
   FPS: 6
   Decode chunk size: 2
   حافظه GPU: 5.6 GB


In [13]:
# سلول 2: توابع مدیریت حافظه
def clear_memory():
    """پاک کردن حافظه GPU و CPU"""
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.synchronize()

def get_memory_info():
    """نمایش اطلاعات حافظه"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        cached = torch.cuda.memory_reserved() / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        
        print(f"🧠 حافظه GPU:")
        print(f"   تخصیص یافته: {allocated:.2f} GB")
        print(f"   کش شده: {cached:.2f} GB")
        print(f"   کل: {total:.2f} GB")
        print(f"   درصد استفاده: {(allocated/total)*100:.1f}%")
        
        return allocated, cached, total
    return 0, 0, 0

def optimize_torch_settings():
    """بهینه‌سازی تنظیمات PyTorch"""
    # تنظیمات حافظه
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cudnn.benchmark = True
    
    # تنظیمات deterministic برای کاهش مصرف حافظه
    torch.use_deterministic_algorithms(False)
    
    # تنظیمات garbage collection
    torch.cuda.empty_cache()
    
    # تنظیم حد مصرف حافظه
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory
        memory_fraction = config.MAX_MEMORY_ALLOCATED
        torch.cuda.set_per_process_memory_fraction(memory_fraction)
        print(f"✅ حد مصرف حافظه: {memory_fraction*100}%")

optimize_torch_settings()


✅ حد مصرف حافظه: 80.0%


In [14]:
# سلول 3: لود مدل بهینه‌سازی شده
def load_optimized_model():
    """لود مدل با بهینه‌سازی حافظه"""
    print("🔄 در حال لود مدل SVD-XT با بهینه‌سازی حافظه...")
    
    clear_memory()
    
    try:
        from diffusers import StableVideoDiffusionPipeline
        
        # لود pipeline با تنظیمات بهینه
        pipe = StableVideoDiffusionPipeline.from_pretrained(
            "stabilityai/stable-video-diffusion-img2vid-xt",
            torch_dtype=torch.float16,
            variant="fp16",
            use_safetensors=True,
            low_cpu_mem_usage=True
        )
        
        # بهینه‌سازی‌های حافظه
        if config.ENABLE_CPU_OFFLOAD:
            pipe.enable_model_cpu_offload()
            print("✅ CPU offloading فعال شد")
        
        if config.ENABLE_SEQUENTIAL_OFFLOAD:
            try:
                pipe.enable_sequential_cpu_offload()
                print("✅ Sequential CPU offload فعال شد")
            except Exception as e:
                print(f"⚠️ Sequential offload خطا: {e}")
        
        if config.ENABLE_ATTENTION_SLICING:
            pipe.enable_attention_slicing(1)  # slice_size=1 برای کمترین مصرف
            print("✅ Attention slicing فعال شد")
        
        # بهینه‌سازی VAE
        try:
            pipe.vae.enable_slicing()
            print("✅ VAE slicing فعال شد")
        except:
            print("⚠️ VAE slicing در دسترس نیست")
        
        # بهینه‌سازی UNet
        try:
            pipe.unet.enable_forward_chunking(chunk_size=1)
            print("✅ UNet forward chunking فعال شد")
        except:
            print("⚠️ UNet chunking در دسترس نیست")
        
        # تنظیمات low VRAM
        if config.LOW_VRAM_MODE:
            try:
                pipe.enable_vae_slicing()
                pipe.enable_vae_tiling()
                print("✅ VAE tiling فعال شد")
            except:
                print("⚠️ VAE tiling در دسترس نیست")
        
        print("✅ مدل با بهینه‌سازی حافظه لود شد")
        
        # بررسی حافظه بعد از لود
        get_memory_info()
        
        return pipe
        
    except Exception as e:
        print(f"❌ خطا در لود مدل: {e}")
        clear_memory()
        return None


In [20]:
# سلول 4: تولید ویدیو بهینه‌سازی شده
def generate_optimized_video(pipe, image_path, output_path="optimized_video.mp4"):
    """تولید ویدیو با مدیریت حافظه"""
    
    print("🎬 شروع تولید ویدیو بهینه‌سازی شده...")
    
    try:
        from diffusers.utils import load_image
        from PIL import Image
        import numpy as np
        import imageio
        
        # پاک کردن حافظه قبل از شروع
        clear_memory()
        
        # لود و آماده‌سازی عکس
        print("📸 لود عکس...")
        image = load_image(image_path)
        
        # تغییر اندازه به رزولیوشن کمتر برای کاهش مصرف حافظه
        optimal_width, optimal_height = 768, 432  # کاهش از 1024x576
        image = image.resize((optimal_width, optimal_height), Image.LANCZOS)
        
        # تنظیم generator
        generator = torch.manual_seed(config.SEED) if config.SEED else None
        
        print(f"🔄 تولید {TOTAL_FRAMES} فریم...")
        print("⏳ این فرآیند ممکن است چند دقیقه طول بکشد...")
        
        # تولید فریم‌ها با تنظیمات بهینه
        with torch.no_grad():  # کاهش مصرف حافظه
            frames = pipe(
                image,
                decode_chunk_size=config.DECODE_CHUNK_SIZE,
                generator=generator,
                motion_bucket_id=config.MOTION_BUCKET_ID,
                noise_aug_strength=config.NOISE_AUG_STRENGTH,
                num_frames=TOTAL_FRAMES,
                num_inference_steps=20,  # کاهش از 25 به 20
                output_type="pil"
            ).frames[0]
        
        print(f"✅ {len(frames)} فریم تولید شد")
        
        # پاک کردن حافظه بعد از تولید
        clear_memory()
        
        # Upscaling به رزولیوشن نهایی
        if config.OUTPUT_WIDTH != optimal_width or config.OUTPUT_HEIGHT != optimal_height:
            print("🔄 تغییر اندازه فریم‌ها...")
            upscaled_frames = []
            
            for i, frame in enumerate(frames):
                # تغییر اندازه فریم به فریم
                frame_resized = frame.resize(
                    (config.OUTPUT_WIDTH, config.OUTPUT_HEIGHT), 
                    Image.LANCZOS
                )
                upscaled_frames.append(frame_resized)
                
                # پاک کردن حافظه هر 5 فریم
                if i % 5 == 0:
                    clear_memory()
                    print(f"   فریم {i+1}/{len(frames)} پردازش شد")
            
            frames = upscaled_frames
            del upscaled_frames
        
        # تبدیل به numpy array
        print("🎥 ذخیره ویدیو...")
        frames_array = []
        
        for i, frame in enumerate(frames):
            frames_array.append(np.array(frame))
            
            # پاک کردن حافظه هر 3 فریم
            if i % 3 == 0:
                clear_memory()
        
        # ذخیره ویدیو
        imageio.mimsave(
            output_path,
            frames_array,
            fps=config.FPS,
            quality=7,  # کاهش کیفیت برای فایل کوچکتر
            codec='libx264',
            macro_block_size=16
        )
        
        print(f"✅ ویدیو ذخیره شد: {output_path}")
        
        # محاسبه حجم فایل
        file_size = os.path.getsize(output_path) / (1024 * 1024)
        print(f"📊 حجم فایل: {file_size:.2f} MB")
        
        # پاک کردن حافظه نهایی
        del frames, frames_array
        clear_memory()
        
        return output_path
        
    except torch.cuda.OutOfMemoryError as e:
        print(f"❌ خطای کمبود حافظه CUDA: {e}")
        print("💡 پیشنهادات:")
        print("   - تعداد فریم‌ها را کاهش دهید")
        print("   - رزولیوشن را کمتر کنید")
        print("   - decode_chunk_size را کمتر کنید")
        clear_memory()
        return None
        
    except Exception as e:
        print(f"❌ خطای عمومی: {e}")
        clear_memory()
        return None


In [21]:
# سلول 5: تست سریع
def quick_test():
    """تست سریع برای بررسی عملکرد"""
    print("🧪 تست سریع حافظه...")
    
    # بررسی حافظه موجود
    allocated, cached, total = get_memory_info()
    
    if total == 0:
        print("❌ CUDA در دسترس نیست")
        return False
    
    # بررسی حافظه آزاد
    free_memory = total - allocated
    required_memory = 3.0  # GB تقریبی مورد نیاز
    
    if free_memory < required_memory:
        print(f"⚠️ حافظه کافی نیست. آزاد: {free_memory:.1f}GB، مورد نیاز: {required_memory}GB")
        print("💡 تنظیمات پیشنهادی:")
        print(f"   - DURATION_SECONDS = {max(1, config.DURATION_SECONDS-1)}")
        print(f"   - OUTPUT_WIDTH = {config.OUTPUT_WIDTH//2}")
        print(f"   - OUTPUT_HEIGHT = {config.OUTPUT_HEIGHT//2}")
        return False
    
    print(f"✅ حافظه کافی است. آزاد: {free_memory:.1f}GB")
    return True

# اجرای تست
test_result = quick_test()

🧪 تست سریع حافظه...
🧠 حافظه GPU:
   تخصیص یافته: 0.01 GB
   کش شده: 0.02 GB
   کل: 5.65 GB
   درصد استفاده: 0.1%
✅ حافظه کافی است. آزاد: 5.6GB


In [22]:
# سلول 6: اجرای نهایی
def run_optimized_generation(image_path):
    """اجرای کامل تولید ویدیو بهینه‌سازی شده"""
    
    # بررسی نهایی حافظه
    if not quick_test():
        return None
    
    # لود مدل
    pipe = load_optimized_model()
    if pipe is None:
        return None
    
    # تولید ویدیو
    result = generate_optimized_video(pipe, image_path)
    
    # آزاد کردن مدل از حافظه
    del pipe
    clear_memory()
    
    return result

print("\n🚀 کد بهینه‌سازی شده آماده است!")
print("📝 برای استفاده:")
print("   result = run_optimized_generation('path/to/your/image.jpg')")


🚀 کد بهینه‌سازی شده آماده است!
📝 برای استفاده:
   result = run_optimized_generation('path/to/your/image.jpg')


In [23]:
result = run_optimized_generation('pics/Marcello_Bacciarelli_-_Alcibiades_Being_Taught_by_Socrates,_1776-77_crop.jpg')

🧪 تست سریع حافظه...
🧠 حافظه GPU:
   تخصیص یافته: 0.01 GB
   کش شده: 0.02 GB
   کل: 5.65 GB
   درصد استفاده: 0.1%
✅ حافظه کافی است. آزاد: 5.6GB
🔄 در حال لود مدل SVD-XT با بهینه‌سازی حافظه...


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

✅ CPU offloading فعال شد
✅ Sequential CPU offload فعال شد
✅ Attention slicing فعال شد
⚠️ VAE slicing در دسترس نیست
✅ UNet forward chunking فعال شد
⚠️ VAE tiling در دسترس نیست
✅ مدل با بهینه‌سازی حافظه لود شد
🧠 حافظه GPU:
   تخصیص یافته: 0.01 GB
   کش شده: 0.02 GB
   کل: 5.65 GB
   درصد استفاده: 0.1%
🎬 شروع تولید ویدیو بهینه‌سازی شده...
📸 لود عکس...
🔄 تولید 18 فریم...
⏳ این فرآیند ممکن است چند دقیقه طول بکشد...


  0%|          | 0/20 [00:00<?, ?it/s]

✅ 18 فریم تولید شد
🔄 تغییر اندازه فریم‌ها...
   فریم 1/18 پردازش شد
   فریم 6/18 پردازش شد
   فریم 11/18 پردازش شد
   فریم 16/18 پردازش شد
🎥 ذخیره ویدیو...
✅ ویدیو ذخیره شد: optimized_video.mp4
📊 حجم فایل: 2.54 MB
